[Chaining, or Why You Should Stop Returning Void](http://jamie-wong.com/2012/02/01/stop-returning-void/)

> The unfortunate reality of chaining in statically typed languages is that it doesn’t play nice with inheritance.

Or is it?

[Method chaining + inheritance don't play well together?](https://stackoverflow.com/questions/551263/method-chaining-inheritance-dont-play-well-together)


In [1]:
#include <iostream>

#ifndef METHODCHAININGTEST
#define METHODCHAININGTEST
struct Point{
  Point():
    x(0),
    y(0){}
  Point(double _x, double _y):
    x(_x),
    y(_y){}
  double x,y;
};
struct Image{
  void setAngle(double angle){}  
};
// member data omitted for brevity

// assume that "setAngle" needs to be implemented separately
// in Label and Image, and that Button does need to inherit
// Label, rather than, say, contain one (etc)
struct Widget {
    Widget(){}
    Widget& move(Point newPos) { pos = newPos; return *this; }
    Point pos;
};

struct Label : Widget {
    Label(){}
    Label& setText(std::string const& newText) { text = newText; return *this; }
    Label& setAngle(double newAngle) { angle = newAngle; return *this; }
    std::string text;
    double angle;
};

    
struct Button : Label {
    Button(){}
    Image backgroundImage;
    Button& setAngle(double newAngle) {
        backgroundImage.setAngle(newAngle);
        Label::setAngle(newAngle);
        return *this;
    }
};
#endif
[]()
{
auto run_add_err = []()
{

Button btn;

// oops: Widget::setText doesn't exist
btn.move(Point(0,0)).setText("Hey");

// oops: calling Label::setAngle rather than Button::setAngle
btn.setText("Boo").setAngle(.5); 

};
run_add_err();
}();

input_line_8:49:22: error: no member named 'setText' in 'Widget'
btn.move(Point(0,0)).setText("Hey");
~~~~~~~~~~~~~~~~~~~~ ^


Interpreter Error: 

In [4]:
#include <iostream>

#ifndef CRTPTEST
#define CRTPTEST
template <typename Q, typename T>
class Default {
public:
    typedef Q type;
};

template <typename T>
class Default<void, T> {
public:
    typedef T type;
};
    
class CRTPPoint{
public:
  CRTPPoint():
    x(0),
    y(0){}
  CRTPPoint(double _x, double _y):
    x(_x),
    y(_y){}
  double x,y;
};
class CRTPImage{
public:
  void setAngle(double angle){}  
};


template<class T>
class CRTPWidget {
public:
    typedef typename Default<T, CRTPWidget<void> >::type type;
    CRTPWidget(){}
    type& move(CRTPPoint newPos) { pos = newPos; return static_cast<type&>(*this); }
    CRTPPoint pos;
};

template<class T=void>
class CRTPLabel : public CRTPWidget<CRTPLabel<T> > {
public:
     typedef typename Default<T, CRTPWidget<CRTPLabel<T> > >::type type;
    CRTPLabel(){}
    type& setText(std::string const& newText) { text = newText; return static_cast<type&>(*this); }
    type& setAngle(double newAngle) { angle = newAngle; return static_cast<type&>(*this); }
    std::string text;
    double angle;
};

template<class T=void>
class CRTPButton : public CRTPLabel<CRTPButton<T> > {
public:
    typedef typename Default<T, CRTPLabel<CRTPButton<T> > >::type type;

    CRTPButton(){}
    CRTPImage backgroundImage;
    type& setAngle(double newAngle) {
        backgroundImage.setAngle(newAngle);
        CRTPLabel<CRTPButton<T> >::setAngle(newAngle);
        return static_cast<type&>(*this);
    }
};
#endif



In [5]:
[]()
{
auto run_add_err = []()
{

CRTPButton<> btn;

 btn.move(CRTPPoint(0,0)).setText("Hey");

 btn.setText("Boo").setAngle(.5); 

};
run_add_err();
}();